# Storage Management

- Linux systems follow the Filesystem Heirarchy Standard [Filesystem Heirarchy Standard](https://en.wikipedia.org/wiki/Filesystem_Hierarchy_Standard)
- There are a variety of File System types supported in Linux:
  - Disk based
  - Network based
  - Memory based (aka pseudo file systems like /proc)

- A summary of Linux directories:
  - /etc (extended text configuration, not etcetera...):  holds system configuration files
  - /root: the home directory for the root user
  - /mnt: this directory is used to mount file systems temporarily
  - /boot: contains the Linux kernel, boot support files, and boot config files
  - /home: contains user home directories
  - /opt: this directory can be used to hold optional software
  - /usr (unix system resource): This directory contains most of the system files. It has some subdirectories:
    - /usr/bin: contains binaries
    - /usr/sbin: binaries that require root user privileges
    - /usr/lib and /usr/lib64: contain shared library routines required by other commands and programs
    - /usr/local: serves as a system administration repository for storing tools
    - /usr/share: contains manual pages, docs, templates, etc.
    - /usr/src: this directory is used to store source code
  - /var: contains data that frequently changes (such as log files)
  - /tmp: stores temporary files. Auto removed if not accessed after 10 days
  - /dev: stores physical hardware represented as files. These files are managed by the udevd services
  - /proc: contains info about the current state of the running kernel
  - /run: contains data for processes currently running on the system
  - /sys: contains info about hardware devices, drivers. This info is used by the kernel to load devices and drivers. 

- List, Create, and, Modify Storage
  - `lsblk` can be used to list partitions
    ```
    [azureadmin@centos01 /]$ lsblk
    NAME    MAJ:MIN RM  SIZE RO TYPE MOUNTPOINT
    sda       8:0    0   30G  0 disk
    ├─sda1    8:1    0  500M  0 part /boot
    ├─sda2    8:2    0   29G  0 part /
    ├─sda14   8:14   0    4M  0 part
    └─sda15   8:15   0  495M  0 part /boot/efi
    sdb       8:16   0   64G  0 disk
    └─sdb1    8:17   0   64G  0 part /mnt/resource
    ```
  - All block devices can be found in the `/dev/` directory
  - `sudo fdisk -l /dev/sda` can be used to view detailed info about partitions on a block device
  - `cfdisk` can be used to creat a partition on a disk, and is easier to use that `fdisk` 
    - `cfdisk` can be used to resize existing partitions

- Create and Manage Swap Space
  - swap is used when no RAM is available
  - swapping is also known as paging
  - swapping is almost always slower than working with RAM
  - `mkswap` can be used to create swap space
    - `sudo mkwap /dev/sda2`
  - `swapon` can be used to work with swap space
    - `swapon --show`
  - `dd` can be used to write random data to a disk
    - `sudo dd if=/dev/zero of=/dev/sdc2 bs=1M count=128 status=progress`

- Create and Configure File Systems
  - To create a XFS file system:
    - `sudo mkfs.xfs /dev/sdb1`
  - To create a EXT4 file system:
    - `sudo mkfs.ext4 /dev/sdb1`
  - `xfs_admin` can be used to change things on an already created xfs file system
  - `tune2fs` can be used to manage ext file systems (like `xfs_admin`)

- Configure Systems to Mount File Systems at Boot Time
  - File systems are typically mounted in `/mnt`
  - To mount a file system:
    - `sudo mount /dev/vdb1 /mnt/vdb1`
  - To unmount a file system:
    - `sudo umount /mnt/vdb1`
  - `/etc/fstab` contains persistent mount information for disks
  - `swaplabel` can be used to configure the label of swap partitions
    - `sudo swaplabel -L myLabel /dev/sda1`

- On demand file system mounting
  - Linux can mount file systems on demand, as they are needed
  - `Autofs` can be used to mount file systems on demand
    - `sudo dnf install autofs -y`
    - `sudo systemctl enable autofs`
    - `sudo systemctl start autofs`

  - NFS can be used to mount remote file system
    - `sudo dnf install nfs-utils -y`
    - `sudo systemctl start nfs-server.service`
    - `sudo systemctl enable nfs-server.service` 
    - `/etc/exports` contains file share exports
      ```
      [azureadmin@centos01 sdc1]$ cat /etc/exports
      /etc 127.0.0.1(ro)
      ```
    - after adding an entry to `/etc/exports`, you need to restart the nfs daemon
  
  - Autofs
    - To tell Autofs to mount a share on demand, edit `/etc/auto.master`
      - Add the following to `/etc/auto.master`
        `/shares /etc/auto.shares --timeout=400`
      - edit `/etc/auto.shares` and add the following:
        `mynetworkshare -fstype=auto 127.0.0.1:/etc`
    - 

- Evaluating File System Features and Options
  - `findmnt` can be used to view all mounts
  - you can use `findmnt -t <file system type>` to view files system with a certain type (xfs, ext4, etc.)
    - `findmnt -t xfs`
      ```
      [azureadmin@centos01 shares]$ findmnt -t xfs
      TARGET      SOURCE    FSTYPE OPTIONS
      /           /dev/sda2 xfs    rw,relatime,seclabel,attr2,inode64,logbufs=8,logbsize=32k,noquota
      ├─/boot     /dev/sda1 xfs    rw,relatime,seclabel,attr2,inode64,logbufs=8,logbsize=32k,noquota
      └─/mnt/sdc1 /dev/sdc1 xfs    rw,relatime,seclabel,attr2,inode64,logbufs=8,logbsize=32k,noquota
      ```
    - You can add multiple file system types when using `findmnt -t`:
      ```
      [azureadmin@centos01 shares]$ findmnt -t xfs,ext4
      TARGET          SOURCE    FSTYPE OPTIONS
      /               /dev/sda2 xfs    rw,relatime,seclabel,attr2,inode64,logbufs=8,logbsize=32k,noquota
      ├─/boot         /dev/sda1 xfs    rw,relatime,seclabel,attr2,inode64,logbufs=8,logbsize=32k,noquota
      ├─/mnt/resource /dev/sdb1 ext4   rw,relatime,seclabel
      └─/mnt/sdc1     /dev/sdc1 xfs    rw,relatime,seclabel,attr2,inode64,logbufs=8,logbsize=32k,noquota
      ```
- Manage and Configure LVM Storage
  - LVM Acronyms
    - PV = Physical Volume, real physical storage devices
      - `sudo lvmdiskscan` can be used to list devices that may be used as physical volumes
        ```
        04:52:03 azureadmin@centos01 ~ → sudo lvmdiskscan
        /dev/sda1  [     500.00 MiB]
        /dev/sda2  [      29.02 GiB]
        /dev/sda15 [     495.00 MiB]
        /dev/sdb1  [     <64.00 GiB]
        /dev/sdf   [       5.00 GiB]
        /dev/sdh   [       5.00 GiB]
        /dev/sdi   [       5.00 GiB]
        3 disks
        4 partitions
        0 LVM physical volume whole disks
        0 LVM physical volumes
        ```
      - `'sudo pvcreate /dev/sdc /dev/sdd /dev/sde` can be used to create a new LVM volume from 3 disks
        ```
        04:52:06 azureadmin@centos01 ~ → sudo pvcreate /dev/sdf /dev/sdh /dev/sdi
        Physical volume "/dev/sdf" successfully created.
        Physical volume "/dev/sdh" successfully created.
        Physical volume "/dev/sdi" successfully created.
        ```
      - `sudo pvs` can be used to list physical volumes used by LVM
        ```
        04:52:19 azureadmin@centos01 ~ → sudo pvs
        PV         VG Fmt  Attr PSize PFree
        /dev/sdf      lvm2 ---  5.00g 5.00g
        /dev/sdh      lvm2 ---  5.00g 5.00g
        /dev/sdi      lvm2 ---  5.00g 5.00g
        ```
  
    - VG = Volume Group
      - After LVM has physical devices (pvs), you add the pvs to a volume group (vg). This tells LVM how it can use the storage capacity
      - `sudo vgcreate my_volume /dev/sdf /dev/sdh` will create a new VG with 2 PV
        ```
        04:58:13 azureadmin@centos01 ~ → sudo vgcreate my_vol /dev/sdf /dev/sdh
        Volume group "my_vol" successfully created
        ```
      - Once disks are added to a volume group, they are seen by the system as one contiguous block of storage
      - You can add another disk to the volume group using `vgextend`
        ```
        05:00:06 azureadmin@centos01 ~ → sudo vgextend my_vol /dev/sdi
        Volume group "my_vol" successfully extended
        ```
      - use `sudo vgs` to view the status of Volume Groups:
        ```
        05:00:14 azureadmin@centos01 ~ → sudo vgs
        VG     #PV #LV #SN Attr   VSize   VFree
        my_vol   3   0   0 wz--n- <14.99g <14.99g
        ```
      - use `sudo vgreduce my_vol /dev/sdi` to remove a physical volume from a volume group
        ```
        05:00:39 azureadmin@centos01 ~ → sudo vgreduce my_vol /dev/sdi
        Removed "/dev/sdi" from volume group "my_vol"
        ```

    - LV = Logical Volume
      - A logical volume is similar to a partition
      - `sudo lvcreate --size 2G --name partition1 my_vol` can be used to create a logical volume of 2 gigabytes
        ```
        05:02:22 azureadmin@centos01 ~ → sudo lvcreate --size 2G --name partition1 my_vol
        Logical volume "partition1" created.
        ```
      - You can view logical volumes using `sudo lvs`
        ```
        05:03:49 azureadmin@centos01 ~ → sudo lvs
        LV         VG     Attr       LSize Pool Origin Data%  Meta%  Move Log Cpy%Sync Convert
        partition1 my_vol -wi-a----- 2.00g
        partition2 my_vol -wi-a----- 6.00g
        ```
      - to tell a logical volume to use all space on a logical volume, use `sudo lvresize` 
        ```
        05:03:50 azureadmin@centos01 ~ → sudo lvresize --extents 100%VG my_vol/partition1
        Reducing 100%VG to remaining free space 3.99 GiB in VG.
        Size of logical volume my_vol/partition1 changed from 2.00 GiB (512 extents) to 3.99 GiB (1022 extents).
        Logical volume my_vol/partition1 successfully resized.
        ```
      - the path to LVs on the system can be found using `lvdisplay`
        ```
        05:08:32 azureadmin@centos01 ~ → sudo lvdisplay  | grep "LV Path"
        LV Path                /dev/my_vol/partition1
        LV Path                /dev/my_vol/partition2
        ```
      - You can then add a file system to a LV using common file system management commands
        ```
        05:08:41 azureadmin@centos01 ~ → sudo mkfs.xfs /dev/my_vol/partition1
        meta-data=/dev/my_vol/partition1 isize=512    agcount=4, agsize=261632 blks
                =                       sectsz=4096  attr=2, projid32bit=1
                =                       crc=1        finobt=1, sparse=1, rmapbt=0
                =                       reflink=1
        data     =                       bsize=4096   blocks=1046528, imaxpct=25
                =                       sunit=0      swidth=0 blks
        naming   =version 2              bsize=4096   ascii-ci=0, ftype=1
        log      =internal log           bsize=4096   blocks=2560, version=2
                =                       sectsz=4096  sunit=1 blks, lazy-count=1
        realtime =none                   extsz=4096   blocks=0, rtextents=0
        Discarding blocks...Done.
        ```
      - If the LV contains a file system, you must take extra caution when resizing it. You must pass the `--resizefs` parameter to `lvresize`
      - `sudo lvresize --resizefs --size 3G my_vol\partition1`
      - XFS file system shrinking is not supported

  - If you forget what commands to use for LVM, simply open the man pages for LVM and scroll to the bottom to get a list of available commands
    - `man lvm`

  - `sudo lvmdiskscan` will show what disks are available
  - To create a physical volume: 
    - `sudo pvcreate /dev/sdd /dev/sde /dev/sdf`
    - Example:
      ```
      [azureadmin@centos01 shares]$ sudo pvcreate /dev/sdd /dev/sde
      Physical volume "/dev/sdd" successfully created.
      Physical volume "/dev/sde" successfully created.
      ```
    - To list physical volumes: `sudo pvs`
      ```
      [azureadmin@centos01 shares]$ sudo pvs
      PV         VG Fmt  Attr PSize PFree
      /dev/sdd      lvm2 ---  5.00g 5.00g
      /dev/sde      lvm2 ---  5.00g 5.00g
      ```
    - After creating the physcial voume, add it to a volume group:
      - `sudo vgcreate my_volume /dev/sdd /dev/sde`
    - List volume groups:
      ```
      [azureadmin@centos01 shares]$ sudo vgs
      VG        #PV #LV #SN Attr   VSize VFree
      my_volume   2   0   0 wz--n- 9.99g 9.99g
      ```
    - To expand a volume group, add a PV to the physical volume. Then use `vgextend` to add the PV to the volume group
      - `sudo vgextend my_volume /dev/sdf`
    - You can also remove a physical volume from the volume group:
      - `sudo vgreduce my_volume /dev/sdf`
    - Then you can remove the physical volume:
      - `sudo pvremove /dev/sdf`
    - Logical volumes are like partitions
    - you can create a new logical volume:
      - `sudo lvcreate --size 3G --name partition1 my_volume`
    - To grow a logical volume to use all the space it has available
      - `suod lvresize --extents 100%VG my_volume/partition1`

- Encrypted Storage
  - use `cryptsetup --verify-passphrase open --type plain /dev/vde mysecuredisk` to encrypt a disk using plain mode
  - To use LUKS encryption, which is much easier to use:
    - `sudo cryptsetup luksFormat /dev/vde`
      ```
      05:19:37 azureadmin@centos01 ~ → sudo cryptsetup luksFormat /dev/sdi

      WARNING!
      ========
      This will overwrite data on /dev/sdi irrevocably.

      Are you sure? (Type 'yes' in capital letters): YES
      Enter passphrase for /dev/sdi:
      Verify passphrase:
      ```
    - You can change the encryption key easily using LUKS: `sudo cryptsetup luksChangekey /dev/vde`

- RAID
  - L0 / Stripe
  - L1 / Mirror
  - L5 / Mirror w/ parity
  - L10 / L1 + L0
  - `mdadm --create /dev/md0 --level=0 --raid-devices=3 /dev/vdc /dev/vdd /dev/vde` can be used to create a L0 array with 3 disks


- ACLs
  - ACLs are more granular that UGO/RWX permissions
  - Can be applied per user
  - `setfacl` can be used to set an ACL on a file or directory
    - `sudo setfacl -m user:azureadmin:rw examplefile`
  - `getfacl` can be used to list ACLs on a file or directory
    ```
    09:45:30 azureadmin@centos01 ~ → sudo getfacl examplefile
    # file: examplefile
    # owner: adm
    # group: ftp
    user::rw-
    user:azureadmin:rw-
    group::rw-
    mask::rw-
    other::r--
    ```
  - `setfacl` can change files recursively in a directory
    - `sudo setfacl --recursive -m user:azureadmin:rwx testrecursive/`
    - `sudo setfacl --remove --remove user:azureadmin testrecursive/`

  - the **+** at the end of the permissions in the output below means this file has an ACL applied
    ```
    09:45:21 azureadmin@centos01 ~ → ll examplefile
    rw-rw-r--+ 1 adm ftp 46 Sep 18 09:45 examplefile
    ```
- Attributes
  - `chattr` can be used to change attributes on a file
  - immutable attribute - file cannot be changed in any way
  - appendonly attribute - file cannot be written to
  - `lsattr` can be used to list attributes on a file or directory

- Commands
```
tree
ls
pwd

mdadm

setfacl
getfacl

chattr
lsattr

```

